In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import numpy as np

# Обработка данных

In [2]:
df = pd.read_csv('lenta-ru-news.csv');

C:\Users\Dimka\AppData\Local\Temp\ipykernel_22704\2058247269.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lenta-ru-news.csv');


In [3]:
df.shape

(800975, 6)

In [7]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [8]:
df = df.drop(['url', 'topic', 'date'], axis=1)

In [9]:
df = df[df['text'].notnull()]

In [30]:
list(map(lambda x: snowball.stem(x), ['бои', 'мама']))

['бо', 'мам']

In [33]:
stop_words = stopwords.words('russian')
snowball = SnowballStemmer(language='russian')

def upd_text(text):
    text = text.lower()
    words = re.findall(r'\w+', text)
    filter_words = list(filter(lambda x: x not in stop_words, words))
    if filter_words:
        # new = list(map(lambda x: snowball.stem(x), filter_words))
        new = list(np.vectorize(snowball.stem)(filter_words))
        return ' '.join(new)
    return ''


In [39]:
from multiprocessing import Pool
import numpy as np

n_cores = 3
pool = Pool(n_cores)

def apply_parallel(df, func):
    df_split = np.array_split(df, n_cores)
    df = pd.concat(pool.map(func, df_split))
    return df

In [36]:
upd = apply_parallel(df['text'], upd_text)
upd

0       бо сопоцкин друскеник законч отступлен германц...
1       министерств народн просвещен вид происходя чре...
2       штабс капита п н нестер днях увид район желтие...
3       фотограф корреспондент daily mirror рассказыва...
4       лиц приеха варшав люблин переда туд доставл шв...
                              ...                        
9995    глав мвд дагеста приказа наряд милиц пригранич...
9996    представител кремл ведут активн диалог губерна...
9997    москв 20 июл открыва фестивал планет клоунад с...
9998    чист американск получ фина уимблдонск теннисн ...
9999    перв российск клуб стартова сезон 2000 2001 го...
Name: text, Length: 10000, dtype: object

In [7]:
df['upd'] = upd

In [10]:
df.to_csv('upd1.csv')

# Предварительнй рассчет инвертированного индекса

In [ ]:
df = pd.read_csv('upd1.csv')

inv_index = {}
for idx, text in enumerate(df['upd']):
    for word in text.split():
        if word not in inv_index:
            inv_index[word] = []
        inv_index[word] = [idx]

In [ ]:
import pickle


with open('inv_index.pkl', 'wb') as f:
    pickle.dump(inv_index, f)

# Предварительнй рассчет TF_IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
idf_vectorizer = TfidfVectorizer()

corpus = df['upd'].tolist()
Y = idf_vectorizer.fit_transform(corpus)

In [ ]:
with open('TF_IDF.pkl', 'wb') as f:
    pickle.dump(Y, f)

In [ ]:
vocab = idf_vectorizer.vocabulary_

with open('TF_IDF_vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

# Предварительнй рассчет списка документов

In [8]:
from my_search import Document
import pickle

df = pd.read_csv('upd1.csv')
documents = []
for idx, text in enumerate(df['upd']):
    documents.append(Document(df['title'][idx], df['text'][idx], idx))

In [9]:
with open('documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

In [11]:
len(documents)

800970